**кластеризатор текстов**

SnowballStemmer + TFIDF + DBSCAN

_Евгений Борисов <esborisov@sevsu.ru>_

## тексты

In [1]:
import pandas as pd
pd.options.display.precision = 2 
pd.options.display.max_colwidth = 1000 

from tqdm.notebook import tqdm
tqdm.pandas()

In [2]:
# загружаем тексты
data = pd.read_pickle('../data/news.pkl.gz')
print('записей:',len(data))
data.sample(2)

записей: 3196


,text,tag
1420,"Избранный президент США Дональд Трамп заявил, что покинет свои компании до 20 января.\n\n«Хоть закон и не обязывает меня это делать, я оставлю бизнес до 20 января, чтобы сосредоточиться на президентстве. Двое моих детей, Дон и Эрик, а также члены руководства будут им управлять», — написал Трамп в своём Twitter.\n\nОн отметил, что никаких новых сделок не будет заключено во время его президентского срока (сроков).\n\nEven though I am not mandated by law to do so, I will be leaving my busineses before January 20th so that I can focus full time on the...... — Donald J. Trump (@realDonaldTrump) December 13, 2016\n\nPresidency. Two of my children, Don and Eric, plus executives, will manage them. No new deals will be done during my term(s) in office. — Donald J. Trump (@realDonaldTrump) December 13, 2016\n\nРанее сообщалось, что Дональд Трамп стал человеком года по версии Financial Times.",politics
691,"Журналист против ""Милкавиты"": автор статьи о радиоактивном молоке предложил мировое соглашение\n\n9 декабря 2016 в 18:30\n\nTUT.BY\n\nВ Экономическом суде Минска продолжают рассматривать дело журналиста агентства Associated Press Юрася Карманова против гомельского завода «Милкавита». На очередном заседании Карманов предложил предприятию заключить мировое соглашение.\n\nФото: Дарья Бурякина, TUT.BY\n\nНапомним, в апреле журналисты Associated Press из поездки по чернобыльской зоне привезли молоко фермера из Хойникского района (молоко у него покупала «Милкавита») и сдали его на анализ в Минский городской центр гигиены и эпидемиологии. Результаты оказались неутешительны: показатели по стронцию-90 в десять раз превышали норму.\n\nЮрась Карманов пояснил TUT.BY, что для него в суде было важно доказать: сведения о цезии и стронции он получил от представителей госоргана.\n\n— В суд пришла представительница Минского городского центра гигиены и эпидемиологии и подтвердила: это они выдали нам ...",social


In [3]:
len( data.drop_duplicates('text') )

3196

## токенайзер

In [5]:
# import re
# from nltk.tokenize import word_tokenize as nltk_tokenize_word

# def tokenizer(text):
#     return [
#             t for t in nltk_tokenize_word( # разбиваем текст на слова
#                 re.sub(r'</?[a-z]+>',' ',text), # удаляем xml tag 
#                 language='russian'
#             ) 
#         ]

In [6]:
# import re
# from nltk.tokenize import word_tokenize as nltk_tokenize_word
# from nltk.corpus import stopwords as nltk_stopwords

# stopwords = set(nltk_stopwords.words('russian'))

# def tokenizer(text,stopwords=stopwords):
#     return [
#             t for t in nltk_tokenize_word( # разбиваем текст на слова
#                 re.sub(r'</?[a-z]+>',' ',text), # удаляем xml tag 
#                 language='russian'
#             ) 
#             if not (
#                False
#                or (len(t)<3) # выкидываем очень короткие слова
#                or re.match(r'^[^a-zA-ZЁёА-я]+$', t) # выкидываем токены не содержащие букв
#                or re.match(r'^(\w)\1+$', t)  # выкидываем токены из одного повторяющегося символа
#                or re.match(r'^[^a-zA-ZЁёА-я].*$', t)  # выкидываем токены начинающиеся не с буквы
#                or (t in stopwords) # выкидываем предлоги, союзы и т.п.    
#             )
#         ] 

In [12]:
import re
# from razdel import sentenize
from razdel import tokenize
from nltk.corpus import stopwords as nltk_stopwords
stopwords = set(nltk_stopwords.words('russian'))

def tokenizer(text,stopwords=stopwords):
    return [
            t.text for t in tokenize( # разбиваем текст на слова
                re.sub(r'</?[a-z]+>',' ',text), # удаляем xml tag 
            ) 
            if not (
               False
               or (len(t.text)<3) # выкидываем очень короткие слова
               or re.match(r'^[^a-zA-ZЁёА-я]+$', t.text) # выкидываем токены не содержащие букв
               or re.match(r'^(\w)\1+$', t.text)  # выкидываем токены из одного повторяющегося символа
               or re.match(r'^[^a-zA-ZЁёА-я].*$', t.text)  # выкидываем токены начинающиеся не с буквы
               or (t.text in stopwords) # выкидываем предлоги, союзы и т.п.    
            )
        ] 

## выполняем частотный анализ

In [13]:
# from sklearn.feature_extraction.text import CountVectorizer
# tf_model = CountVectorizer(
#         min_df=.01, # выкидываем очень редкие слова
#         max_df=.25, # выкидываем очень частые слова
#         tokenizer=tokenizer, # ф-ция токенайзер
#         token_pattern=None, # отключаем дефолтный токенайзер
#         binary=True,
#     )

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf_model = TfidfVectorizer(
        min_df=.01, # выкидываем очень редкие слова
        max_df=.10, # выкидываем очень частые слова
        use_idf=False, # не используем обратную частоту
        norm='l2', # нормируем TF
        tokenizer=tokenizer, # ф-ция токенайзер
        token_pattern=None, # отключаем дефолтный токенайзер
    )

In [15]:
%%time

data_tf = tf_model.fit_transform( data['text'] )

display(data_tf.shape)

(3196, 2025)

CPU times: user 5.58 s, sys: 7.02 ms, total: 5.59 s
Wall time: 5.63 s


In [16]:
vocab = sorted(tf_model.vocabulary_)
display(len(vocab))
display(vocab)

2025

['adobe',
 'afisha',
 'apple',
 'audi',
 'auto',
 'com',
 'facebook',
 'finance',
 'flash',
 'google',
 'html',
 'http',
 'javascript',
 'journal',
 'kia',
 'lenta',
 'news',
 'player',
 'realty',
 'regnum',
 'reuters',
 'sport',
 'street',
 'telegram',
 'the',
 'times',
 'twitter',
 'volkswagen',
 'wall',
 'youtube',
 'абсолютно',
 'аварии',
 'авария',
 'августа',
 'августе',
 'авто',
 'автобуса',
 'автомобилей',
 'автомобилем',
 'автомобили',
 'автомобиль',
 'автомобиля',
 'автор',
 'авторы',
 'агентства',
 'агентство',
 'администрации',
 'адрес',
 'актер',
 'активно',
 'активность',
 'акции',
 'акций',
 'александр',
 'александра',
 'александром',
 'алексей',
 'алеппо',
 'американская',
 'американские',
 'американский',
 'американских',
 'американского',
 'американской',
 'америки',
 'анализ',
 'аналитики',
 'андрей',
 'андрея',
 'анна',
 'армии',
 'ассоциации',
 'атмосферу',
 'базе',
 'базовой',
 'банк',
 'банка',
 'безопасности',
 'беларуси',
 'беларусь',
 'белорусов',
 'белорусска

## кластеризируем

In [17]:
# оценки расстояний 
from sklearn.metrics.pairwise import euclidean_distances
d = euclidean_distances(data_tf)
d[d>0.].min(),d[d>0.].mean(),d.max(),

(0.06485447940530417, 1.395762969872272, 1.4142135623730976)

In [18]:
%%time

from sklearn.cluster import DBSCAN
data['cluster_id'] = DBSCAN(eps=.7,min_samples=3).fit(data_tf).labels_
data['cluster_id'].drop_duplicates().count()

CPU times: user 160 ms, sys: 49.6 ms, total: 209 ms
Wall time: 210 ms


31

In [19]:
# номер кластера, количество объектов, метки объектов
# (cluster=-1 - некластеризованные DBSCAN объекты) 
cluster_descr = pd.concat([
        data[['cluster_id','tag']].groupby(['cluster_id'])['tag'].count(),
        data[['cluster_id','tag']].groupby(['cluster_id'])['tag'].apply(lambda s: set(s)).apply(' '.join)
    ],axis=1).reset_index()

cluster_descr.columns = ['cluster_id','count','tags']

cluster_descr

,cluster_id,count,tags
0,-1,3065,economics woman social culture incident health sport reclama tech realty science auto politics
1,0,5,tech economics culture health
2,1,12,social culture tech sport
3,2,6,economics
4,3,6,politics
5,4,3,politics
6,5,4,politics
7,6,11,politics
8,7,9,politics
9,8,4,incident


In [20]:
data.query('cluster_id==2')

,text,tag,cluster_id
929,"Доллар и евро подешевели в преддверии выходных\n\n9 декабря 2016 в 12:59\n\nTUT.BY\n\nФото: Дмитрий Брушко, TUT.BY\n\nНа Белорусской валютно-фондовой бирже 9 декабря прошли очередные торги валютами.\n\nКурс доллара снизился на BYN0,0007 - до 1,9739 рубля.\n\nЕвро подешевел на BYN0,0295 - до 2,0967 рубля.\n\nКурс российского рубля вырос на BYN0,0052 - до 3,1207 рубля за 100 российских рублей.\n\nОфициальный курс белорусского рубля по отношению к стоимости корзины иностранных валют укрепился на 0,20%.",economics,2
941,"На торгах 8 декабря рубль укрепился к доллару и ослаб к евро\n\n8 декабря 2016 в 13:17\n\nTUT.BY\n\nНа Белорусской валютно-фондовой бирже 8 декабря прошли очередные торги валютами.\n\nФото: Reuters\n\nКурс доллара упал на 0,22%, или BYN0,0043 — до 1,9746 рубля.\n\nЕвро подорожал на 0,2%, или на BYN0,0042 — до 2,1262 рубля.\n\nКурс российского рубля увеличился на 0,65%, или BYN0,0201 — до 3,1155 рубля.",economics,2
956,"Официальный курс белорусского рубля по отношению к корзине иностранных валют ослаб на 0,36%.\n\nКурс российского рубля увеличился на BYN0,0121 — до 3,0954 рубля.\n\nЕвро подорожал на BYN0,0023 — до 2,122 рубля.\n\nКурс доллара вырос на BYN0,0093 — до 1,9789 рубля.\n\nНа торгах 7 декабря рубль ослаб к трем основным валютам\n\nЕсли вы заметили ошибку в тексте новости, пожалуйста, выделите её и нажмите Ctrl+Enter",economics,2
975,"Российский рубль укрепился на BYN0,0126 и составил 3,0843 белорусского рубля за 100 российских.\n\nЕвро снизился на BYN0,0122 до 2,0897 рубля.\n\nДоллар укрепился на BYN0,0003 до 1,9706 рубля.\n\nНа Белорусской валютно-фондовой бирже 5 декабря прошли очередные торги валютами. Снижение рубля к корзине валют составило 0,08%.\n\nНа торгах 5 декабря рубль сдал к российскому рублю, но окреп к евро\n\nЕсли вы заметили ошибку в тексте новости, пожалуйста, выделите её и нажмите Ctrl+Enter",economics,2
993,"На торгах 2 декабря евро и доллар подорожали\n\n2 декабря 2016 в 12:40\n\nTUT.BY\n\nФото: Дмитрий Брушко, TUT.BY\n\nНа Белорусской валютно-фондовой бирже 2 декабря прошли очередные торги валютами. Рубль снизился на 0,20% к корзине валют.\n\nДоллар укрепился на BYN0,0076 до 1,9703 рубля.\n\nЕвро подорожал сразу на BYN0,0204 до 2,1019 рубля.\n\nРоссийский рубль снизился на BYN0,0054 и составил 3,0717 белорусского рубля за 100 российских.",economics,2
1017,"На торгах 30 ноября доллар и евро заметно выросли\n\n30 ноября 2016 в 12:44\n\nTUT.BY\n\nФото: Дмитрий Брушко, TUT.BY\n\nНа Белорусской валютно-фондовой бирже 30 ноября прошли очередные торги валютами. Рубль снизился к корзине валют на 0,20%.\n\nДоллар укрепился на BYN0,0076 до 1,9795 рубля.\n\nЕвро подорожал на BYN0,0131 до 2,1033 рубля.\n\nРоссийский рубль снизился на BYN0,0019 и составил 3,0343 белорусского рубля за 100 российских.",economics,2
